In [ ]:
from pathlib import Path


def fun_构建所有素材根文件夹(root_path: Path, start_num: int):
    all_ma_root_path = []
    for in_path in root_path.iterdir():
        if in_path.is_dir() and int(in_path.stem) >= start_num:
            all_ma_root_path.append(in_path)

    return all_ma_root_path


# 单个文件夹制作详情图片
from fun_获取素材信息.run_获取素材信息 import run_获取素材信息
from fun_制作数据图.fun_1_制作数据图 import run_制作数据图, OneLineInfoModel
from fun_制作详情 import run_制作文件夹详情, fun_裁剪图片
from fun_制作详情 import MakeXQModel
from fun_图片处理 import fun_图片竖向拼接
from tqdm import tqdm
from fun_制作详情标题.fun_制作详情标题 import fun_制作详情标题
from pathlib import Path
import shutil


def fun_单个素材制作详情(ma_root_path: Path):
    title_pil_width_im_spacing = 130

    ma_path = ma_root_path / ma_root_path.stem
    prev_path = ma_root_path / "预览图"
    effec_path = ma_root_path / "效果图"

    save_path_root = Path(rf"C:\Users\wuweihua\Desktop\批量生成详情\{ma_path.stem}")
    if save_path_root.exists() is True:
        shutil.rmtree(save_path_root.as_posix())

    if save_path_root.exists() is False:
        save_path_root.mkdir(parents=True)

    # 制作素材信息图
    ma_info = run_获取素材信息(ma_path.as_posix())
    ma_info_pil = run_制作数据图(
        [
            OneLineInfoModel(title="素材ID", desc=ma_path.stem),
            OneLineInfoModel(title="素材大小", desc=ma_info.size),
            OneLineInfoModel(title="素材格式", desc=ma_info.material_title),
            OneLineInfoModel(
                title="素材文件", desc=" / ".join(ma_info.material_num)
            ),
            OneLineInfoModel(
                title="* 发货方式", desc="拍下后机器人24小时值守秒发货，仅支持百度网盘下载，无法使用网盘地区请勿购买。"
            ),
            OneLineInfoModel(title="* 样机须知", desc="本店样机均是白膜样机，不包含样机内贴图。"),
        ]
    )
    t = fun_制作详情标题(title="素材信息", desc="Material Info 饭桶设计")
    im = fun_图片竖向拼接(
        [t, ma_info_pil],
        title_pil_width_im_spacing,
        "center",
        (255, 255, 255, 255),
    )
    im = im.convert("RGB")
    im.save(rf"{save_path_root}\info.jpg")

    # 制作预览图
    if prev_path.exists() is True:
        prevw_pil = run_制作文件夹详情(
            MakeXQModel(
                image_dir_path=prev_path.as_posix(),
                use_image=1000,
                sort=True,
                line_num=2,
                line_ratio=1.3,
                material_path=ma_path.as_posix(),
                has_info=True,
                xq_width=1500,
                spacing=20,
                crop_position="center",
                img_list=[],
            )
        )
        t = fun_制作详情标题(title="预览图", desc="Material Preview 预览图和源文件对应")
        for num, x in tqdm(
            list(
                enumerate(
                    fun_裁剪图片(
                        im=fun_图片竖向拼接(
                            [t, prevw_pil],
                            title_pil_width_im_spacing,
                            "center",
                            (255, 255, 255, 255),
                        ),
                        oneline_height=2500,
                    )
                )
            ),
            ncols=100,
            desc="保存效果图图片\t",
        ):
            x = x.convert("RGB")
            x.save(
                rf"{save_path_root.as_posix()}\ylt_{num+1}.jpg",
                quality=80,
            )

    # 制作效果图
    if effec_path.exists() is True:
        effec_pil = run_制作文件夹详情(
            MakeXQModel(
                image_dir_path=effec_path.as_posix(),
                use_image=1000,
                sort=True,
                line_num=2,
                line_ratio=1.3,
                material_path=ma_path.as_posix(),
                has_info=False,
                xq_width=1500,
                spacing=20,
                crop_position="center",
                img_list=[],
            )
        )
        t = fun_制作详情标题(title="效果图", desc="Material Effect 此效果图素材内不提供")
        for num, x in tqdm(
            list(
                enumerate(
                    fun_裁剪图片(
                        im=fun_图片竖向拼接(
                            [t, effec_pil],
                            title_pil_width_im_spacing,
                            "center",
                            (255, 255, 255, 255),
                        ),
                        oneline_height=2500,
                    )
                )
            ),
            ncols=100,
            desc="保存效果图图片\t",
        ):
            x = x.convert("RGB")
            x.save(
                rf"{save_path_root.as_posix()}\xgt_{num+1}.jpg",
                quality=80,
            )

In [ ]:
root_path = Path(r"G:\饭桶设计\2000-2999")
start_num = 2678


for in_path in tqdm(
    fun_构建所有素材根文件夹(root_path=root_path, start_num=start_num),
    ncols=100,
    desc="文件夹进度\t",
):
    in_path: Path
    if len(list(in_path.iterdir())) >= 2:
        print(in_path)
        fun_单个素材制作详情(in_path)